In [1]:
# !pip install langchain==0.0.346
# !pip install newspaper3k==0.2.8
# !pip install pandas==2.1.3
# !pip install deltalake==0.14.0
# !pip install tabulate==0.9.0
# !pip install googlenews
# !pip install gnews==0.3.6
# !pip install pyaml==23.9.7

In [2]:
import sys
sys.path.append('..')

# from langchain.document_loaders import NewsURLLoader
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from GoogleNews import GoogleNews
from gnews import GNews

import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 2000)
# pd.set_option('display.expand_frame_repr', False)

from datetime import datetime, timedelta
import logging
import pytz

from tabulate import tabulate

from deltalake import DeltaTable
from deltalake.writer import write_deltalake

import yaml


[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
with open('../config.yml', 'r') as file:
    config = yaml.safe_load(file)

query = config['query']
days_lookback = config['days_lookback']
max_results = config['max_results']

config

{'query': 'Eli Lilly Ozempic Risks', 'days_lookback': 90, 'max_results': 1000}

In [4]:
def print_df(df):
    return tabulate(df, headers='keys', tablefmt='html')

def convert_url_to_parsed_article(url):
    try:
        article = google_news.get_full_article(url)
        article.nlp()
        return article
    except Exception as e:
        logging.info(f'Error within get_full_article: {e}')
        
        return None

def convert_article_to_text(article):
    try:
        return article.text
    except:
        return None

def convert_article_to_html(article):
    try:
        return article.html
    except:
        return None

def convert_article_to_summary(article):
    try:
        return article.summary
    except:
        return None
    
def convert_article_to_keywords(article):
    try:
        return article.keywords
    except:
        return None
    


In [5]:
# Params
table_path = '../data/articles-ingest'
# --------------------------------------------

# query = "Cybertruck"
# days_lookback = 7
# max_results = 1000
current_datetime = datetime.now().astimezone(pytz.utc)
current_date = current_datetime.date()
start_date = (current_datetime - timedelta(days=days_lookback)).date()

# --------------------------------------------

start_date_str = str(start_date.strftime("%Y-%m-%d"))
current_date_str = str(current_date.strftime("%Y-%m-%d"))

print(
    f"""
    end_date: {current_date}
    start_date: {start_date}
    
    
    days_lookback: {days_lookback}
    max_results: {max_results}
    """
)


    end_date: 2023-12-13
    start_date: 2023-09-14
    
    
    days_lookback: 90
    max_results: 1000
    


In [6]:
exclude_websites = ['reuters.com','thestreet.com','digitalchew.com']

google_news = GNews(language='en', 
                    country='US', 
                    start_date=start_date, 
                    end_date=current_date, 
                    # period='7d', 
                    max_results=max_results, 
                    exclude_websites=exclude_websites,
                    # proxy=proxy
                    )

article_results = google_news.get_news(query)
# article_results

In [7]:
df = pd.DataFrame(data=article_results)
df['query'] = query
df['query end date'] = current_datetime
df['published datetime'] = df['published date'].apply(lambda x: datetime.strptime(x, '%a, %d %b %Y %H:%M:%S %Z'))
df['published date'] = df['published datetime'].dt.date
df['published date str'] = df['published date'].astype('datetime64[ns]').dt.strftime('%Y-%m-%d').astype(str)


In [8]:
df['article object'] = df['url'].apply(lambda x: convert_url_to_parsed_article(x))

12/13/2023 02:07:38 AM - Error within get_full_article: 'NoneType' object has no attribute 'nlp'


An error occurred while fetching the article: Article `download()` failed with 403 Client Error: Forbidden for url: https://www.wsj.com/health/pharma/the-next-hot-obesity-drug-wont-be-approved-for-years-online-sellers-hawk-it-anyway-e04d7cc3 on URL https://news.google.com/rss/articles/CBMifWh0dHBzOi8vd3d3Lndzai5jb20vaGVhbHRoL3BoYXJtYS90aGUtbmV4dC1ob3Qtb2Jlc2l0eS1kcnVnLXdvbnQtYmUtYXBwcm92ZWQtZm9yLXllYXJzLW9ubGluZS1zZWxsZXJzLWhhd2staXQtYW55d2F5LWUwNGQ3Y2Mz0gEA?oc=5&hl=en-US&gl=US&ceid=US:en


12/13/2023 02:08:08 AM - Error within get_full_article: 'NoneType' object has no attribute 'nlp'


An error occurred while fetching the article: Article `download()` failed with 403 Client Error: Forbidden for url: https://www.wsj.com/health/pharma/those-new-pickleball-courts-thank-ozempic-581899f3 on URL https://news.google.com/rss/articles/CBMiVGh0dHBzOi8vd3d3Lndzai5jb20vaGVhbHRoL3BoYXJtYS90aG9zZS1uZXctcGlja2xlYmFsbC1jb3VydHMtdGhhbmstb3plbXBpYy01ODE4OTlmM9IBAA?oc=5&hl=en-US&gl=US&ceid=US:en


12/13/2023 02:08:11 AM - Error within get_full_article: 'NoneType' object has no attribute 'nlp'
12/13/2023 02:08:11 AM - Error within get_full_article: 'NoneType' object has no attribute 'nlp'


...

An error occurred while fetching the article: Article `download()` failed with 403 Client Error: Max restarts limit reached for url: https://www.forbes.com/sites/brianbushard/2023/09/16/shortage-of-weight-loss-drugs-like-wegovy-and-ozempic-persist-and-could-for-some-years/ on URL https://news.google.com/rss/articles/CBMijAFodHRwczovL3d3dy5mb3JiZXMuY29tL3NpdGVzL2JyaWFuYnVzaGFyZC8yMDIzLzA5LzE2L3Nob3J0YWdlLW9mLXdlaWdodC1sb3NzLWRydWdzLWxpa2Utd2Vnb3Z5LWFuZC1vemVtcGljLXBlcnNpc3QtYW5kLWNvdWxkLWZvci1zb21lLXllYXJzL9IBkAFodHRwczovL3d3dy5mb3JiZXMuY29tL3NpdGVzL2JyaWFuYnVzaGFyZC8yMDIzLzA5LzE2L3Nob3J0YWdlLW9mLXdlaWdodC1sb3NzLWRydWdzLWxpa2Utd2Vnb3Z5LWFuZC1vemVtcGljLXBlcnNpc3QtYW5kLWNvdWxkLWZvci1zb21lLXllYXJzL2FtcC8?oc=5&hl=en-US&gl=US&ceid=US:en


12/13/2023 02:08:31 AM - Error within get_full_article: 'NoneType' object has no attribute 'nlp'


An error occurred while fetching the article: Article `download()` failed with 403 Client Error: Forbidden for url: https://www.wsj.com/livecoverage/stock-market-today-dow-jones-11-21-2023/card/ozempic-shortages-to-extend-through-2024-RyF6s44bizjC7X2OmOyj on URL https://news.google.com/rss/articles/CBMiiwFodHRwczovL3d3dy53c2ouY29tL2xpdmVjb3ZlcmFnZS9zdG9jay1tYXJrZXQtdG9kYXktZG93LWpvbmVzLTExLTIxLTIwMjMvY2FyZC9vemVtcGljLXNob3J0YWdlcy10by1leHRlbmQtdGhyb3VnaC0yMDI0LVJ5RjZzNDRiaXpqQzdYMk9tT3lq0gEA?oc=5&hl=en-US&gl=US&ceid=US:en


12/13/2023 02:08:32 AM - Error within get_full_article: 'NoneType' object has no attribute 'nlp'


An error occurred while fetching the article: Article `download()` failed with 403 Client Error: Forbidden for url: https://www.healthcentral.com/condition/obesity/surprising-truth-about-obesity-drug-dropouts on URL https://news.google.com/rss/articles/CBMiXGh0dHBzOi8vd3d3LmhlYWx0aGNlbnRyYWwuY29tL2NvbmRpdGlvbi9vYmVzaXR5L3N1cnByaXNpbmctdHJ1dGgtYWJvdXQtb2Jlc2l0eS1kcnVnLWRyb3BvdXRz0gEA?oc=5&hl=en-US&gl=US&ceid=US:en


In [9]:
df['article text'] = df['article object'].apply(lambda x: convert_article_to_text(x) )
df['article html'] = df['article object'].apply(lambda x: convert_article_to_html(x) )
df['article summary'] = df['article object'].apply(lambda x: convert_article_to_summary(x) )
df['article keywords'] = df['article object'].apply(lambda x: convert_article_to_keywords(x) )
# df = df.drop(columns=['article object'])

In [10]:
df

,title,description,published date,url,publisher,query,query end date,published datetime,published date str,article object,article text,article html,article summary,article keywords
0,Zepbound is Eli Lilly’s new weight loss drug: ...,Zepbound is Eli Lilly’s new weight loss drug: ...,2023-11-09,https://news.google.com/rss/articles/CBMiZmh0d...,"{'href': 'https://www.nbcnews.com', 'title': '...",Eli Lilly Ozempic Risks,2023-12-13 02:07:22.818228+00:00,2023-11-09 08:00:00,2023-11-09,<newspaper.article.Article object at 0x7f58ec4...,A new weight loss drug has been approved that ...,"<!DOCTYPE html><html lang=""en""><head><link hre...",A new weight loss drug has been approved that ...,"[drug, lillys, effects, know, weight, eli, zep..."
1,"What to Know About Zepbound, the New Weight Lo...","What to Know About Zepbound, the New Weight Lo...",2023-11-08,https://news.google.com/rss/articles/CBMiSmh0d...,"{'href': 'https://www.nytimes.com', 'title': '...",Eli Lilly Ozempic Risks,2023-12-13 02:07:22.818228+00:00,2023-11-08 08:00:00,2023-11-08,<newspaper.article.Article object at 0x7f58ec4...,The decision by the Food and Drug Administrati...,"<!DOCTYPE html>\n<html lang=""en"" class="" story...",The decision by the Food and Drug Administrati...,"[drug, know, weight, hit, medications, zepboun..."
2,"Ozempic, Wegovy linked to severe medical condi...","Ozempic, Wegovy linked to severe medical condi...",2023-10-05,https://news.google.com/rss/articles/CBMifGh0d...,"{'href': 'https://www.nbcnews.com', 'title': '...",Eli Lilly Ozempic Risks,2023-12-13 02:07:22.818228+00:00,2023-10-05 07:00:00,2023-10-05,<newspaper.article.Article object at 0x7f58ec6...,"People who take popular drugs for weight loss,...","<!DOCTYPE html><html lang=""en""><head><link hre...","People who take popular drugs for weight loss,...","[drugs, drug, risk, weight, finds, medications..."
3,"Study shows Novo's Ozempic, Wegovy pose risk o...","Study shows Novo's Ozempic, Wegovy pose risk o...",2023-10-06,https://news.google.com/rss/articles/CBMifWh0d...,"{'href': 'https://www.fiercepharma.com', 'titl...",Eli Lilly Ozempic Risks,2023-12-13 02:07:22.818228+00:00,2023-10-06 07:00:00,2023-10-06,<newspaper.article.Article object at 0x7f58ec4...,People who use Novo Nordisk’s popular weight-l...,"<!DOCTYPE html>\n<html lang=""en"" dir=""ltr"" pre...",Previous studies of GLP-1 drugs have noted an ...,"[drugs, risk, pose, small, semaglutide, study,..."
4,"New study ties weight-loss drugs Wegovy, Ozemp...","New study ties weight-loss drugs Wegovy, Ozemp...",2023-10-05,https://news.google.com/rss/articles/CBMiPWh0d...,"{'href': 'https://www.cbc.ca', 'title': 'CBC N...",Eli Lilly Ozempic Risks,2023-12-13 02:07:22.818228+00:00,2023-10-05 07:00:00,2023-10-05,<newspaper.article.Article object at 0x7f58ec3...,Medicines in the same class as Novo Nordisk's ...,"<!DOCTYPE html>\n <html lang=""en"">\n ...","Nearly five in every 1,000 users of semaglutid...","[drugs, ties, serious, conditions, risk, drug,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,Does Ozempic Stop Working Over Time? Why Weigh...,Does Ozempic Stop Working Over Time? Why Weigh...,2023-09-18,https://news.google.com/rss/articles/CBMiSGh0d...,"{'href': 'https://www.nytimes.com', 'title': '...",Eli Lilly Ozempic Risks,2023-12-13 02:07:22.818228+00:00,2023-09-18 07:00:00,2023-09-18,<newspaper.article.Article object at 0x7f58ec1...,Kimmy Meinecke blamed herself when she stopped...,"<!DOCTYPE html>\n<html lang=""en"" class="" story...",Kimmy Meinecke blamed herself when she stopped...,"[meinecke, weight, taking, lose, worth, ozempi..."
73,Ozempic Shortages to Extend Through 2024 - The...,Ozempic Shortages to Extend Through 2024 The ...,2023-11-21,https://news.google.com/rss/articles/CBMiiwFod...,"{'href': 'https://www.wsj.com', 'title': 'The ...",Eli Lilly Ozempic Risks,2023-12-13 02:07:22.818228+00:00,2023-11-21 08:00:00,2023-11-21,None,None,None,None,None
74,Wall Street Breakfast Podcast: Walmart Links O...,Wall S

In [11]:
write_deltalake(
    table_path,
    df.drop(columns=['article object']),
    partition_by=['query', 'published date str'],
    mode='overwrite',
    overwrite_schema=True,
    partition_filters=[
        ('query', '=', query),
        ('published date str', '>=', start_date_str), 
        ('published date str', '<=', current_date_str),
    ]
)

In [12]:
# DeltaTable(table_path).to_pandas()